In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 資料讀取與處理

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sms

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/碩二上/Python與機器學習/midterm/Global YouTube Statistics.csv", encoding='windows-1251')

In [ ]:
df.head()

,rank,Youtuber,subscribers,video views,category,Title,uploads,Country,Abbreviation,channel_type,...,created_year,created_month,created_date,Gross tertiary education enrollment (%),Population,Unemployment rate,Urban_population,Latitude,Longitude,mean_monthly_earnings
0,1,T-Series,2450.0,2.280000e+11,Music,T-Series,20082,India,IN,Music,...,2006.0,Mar,13.0,28.1,1.366418e+09,5.36,471031528.0,20.593684,78.962880,4.782300e+00
1,2,YouTube Movies,1700.0,0.000000e+00,Film & Animation,youtubemovies,1,United States,US,Games,...,2006.0,Mar,5.0,88.2,3.282395e+08,14.70,270663028.0,37.090240,-95.712891,2.500000e-08
2,3,MrBeast,1660.0,2.836884e+10,Entertainment,MrBeast,741,United States,US,Entertainment,...,2012.0,Feb,20.0,88.2,3.282395e+08,14.70,270663028.0,37.090240,-95.712891,2.868500e+00
3,4,Cocomelon - Nursery Rhymes,1620.0,1.640000e+11,Education,Cocomelon - Nursery Rhymes,966,United States,US,Education,...,2006.0,Sep,1.0,88.2,3.282395e+08,14.70,270663028.0,37.090240,-95.712891,4.196900e+00
4,5,SET India,1590.0,1.480000e+11,Shows,SET India,116536,India,IN,Entertainment,...,2006.0,Sep,20.0,28.1,1.366418e+09,5.36,471031528.0,20.593684,78.962880,3.877950e+00


In [ ]:
df['category'] = df['category'].fillna('Other')
df['channel_type'] = df['channel_type'].fillna('Other')

In [ ]:
df.at[236, 'created_year'] = 2006
df.at[236, 'created_month'] = 'Dec'
df.at[236, 'created_date'] = 20

df.at[468, 'created_year'] = 2008
df.at[468, 'created_month'] = 'Sep'
df.at[468, 'created_date'] = 17

df.at[508, 'created_year'] = 2009
df.at[508, 'created_month'] = 'Aug'
df.at[508, 'created_date'] = 22

df.at[735, 'created_year'] = 2013
df.at[735, 'created_month'] = 'May'
df.at[735, 'created_date'] = 20

df.at[762, 'created_year'] = 2017
df.at[762, 'created_month'] = 'Mar'
df.at[762, 'created_date'] = 8

In [ ]:
df['mean_monthly_earnings'] = df[['lowest_monthly_earnings', 'highest_monthly_earnings']].mean(axis=1) / 10**6

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import statsmodels.api as sm
from sklearn.metrics import r2_score

# 訂閱，觀看，影片數量和收入預測

In [ ]:
features = df[['subscribers', 'video views', 'uploads']]
target = df['mean_monthly_earnings']

    # 分割數據為訓練集和測試集，並確保保持類別的均衡
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # 定義數據預處理的管道
preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), ['subscribers', 'video views', 'uploads'])
        ])

    # 初始化線性迴歸模型
model = LinearRegression()

    # 建立管道
pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    # 在訓練集上擬合模型
pipeline.fit(X_train, y_train)

    # 預測測試集
y_pred = pipeline.predict(X_test)
    # 在statsmodels中擬合模型
X_train = sm.add_constant(X_train)  # 添加截距項
sm_model = sm.OLS(y_train, X_train).fit()
    # 評估模型性能
r2 = r2_score(y_test, y_pred)
print(f'R-squared: {r2}')
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

R-squared: 0.2180718309020464
Mean Squared Error: 0.24012208689229494


若是沒有按照類別分類做模型，分數很差，不能用。

# 按照類別分類後做訂閱，觀看，影片數量和收入預測

## 線性回歸模型

In [ ]:
from sklearn.model_selection import cross_val_score
categories = df['category'].unique()

# 初始化一個字典，用於存儲每個類別的模型
models2 = {}

for category in categories:
    # 選擇特定類別的數據
    category_data = df[df['category'] == category]

    # 如果該類別樣本數不足，跳過
    if len(category_data) < 10:
        print(f'Skipping category {category} due to insufficient data.')
        continue

    # 選擇特徵變數和目標變數
    features = category_data[['subscribers', 'video views', 'uploads']]
    target = category_data['mean_monthly_earnings']

    # 分割數據為訓練集和測試集，並確保保持類別的均衡
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # 定義數據預處理的管道
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), ['subscribers', 'video views', 'uploads'])
        ])

    # 初始化線性迴歸模型
    model = LinearRegression()

    # 建立管道
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    # 在訓練集上擬合模型
    pipeline.fit(X_train, y_train)

    # 存儲模型
    models2[category] = pipeline

    # 預測測試集
    y_pred = pipeline.predict(X_test)
    # 在statsmodels中擬合模型
    X_train = sm.add_constant(X_train)  # 添加截距項
    sm_model = sm.OLS(y_train, X_train).fit()
    # 評估模型性能
    r2 = r2_score(y_test, y_pred)
    print(f'R-squared in {category}: {r2}')
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error for {category}: {mse}')

R-squared in Music: 0.7630486600375337
Mean Squared Error for Music: 0.018547693423014586
R-squared in Film & Animation: 0.2231233088651695
Mean Squared Error for Film & Animation: 0.16306305905068705
R-squared in Entertainment: 0.24155633495822482
Mean Squared Error for Entertainment: 0.33775762177390956
R-squared in Education: 0.22332664201224905
Mean Squared Error for Education: 0.03703967172412194
R-squared in Shows: 0.5090030674754216
Mean Squared Error for Shows: 1.4821088596552718
R-squared in Other: -2.4693892363194907
Mean Squared Error for Other: 0.36087895742532944
R-squared in People & Blogs: 0.14969831035482062
Mean Squared Error for People & Blogs: 0.2943966203651466
R-squared in Gaming: -5.426590583026028
Mean Squared Error for Gaming: 0.04266904612892314
R-squared in Sports: -9.07325844186596
Mean Squared Error for Sports: 4.405778556414101
R-squared in Howto & Style: 0.1199726430652921
Mean Squared Error for Howto & Style: 0.008222921268404463
R-squared in News & Polit

根據以上數據選擇 Music,Shows 和 News & Politics

In [ ]:
choose = ["Music","Shows","News & Politics"]
for category in choose:
    # 選擇特定類別的數據
    category_data = df[df['category'] == category]

    # 如果該類別樣本數不足，跳過
    if len(category_data) < 10:
        print(f'Skipping category {category} due to insufficient data.')
        continue

    # 選擇特徵變數和目標變數
    features = category_data[['subscribers', 'video views', 'uploads']]
    target = category_data['mean_monthly_earnings']

    # 分割數據為訓練集和測試集，並確保保持類別的均衡
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # 定義數據預處理的管道
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), ['subscribers', 'video views', 'uploads'])
        ])

    # 初始化線性迴歸模型
    model = LinearRegression()

    # 建立管道
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    # 在訓練集上擬合模型
    pipeline.fit(X_train, y_train)

    # 存儲模型
    models2[category] = pipeline

    # 預測測試集
    y_pred = pipeline.predict(X_test)
    # 在statsmodels中擬合模型
    X_train = sm.add_constant(X_train)  # 添加截距項
    sm_model = sm.OLS(y_train, X_train).fit()
    # 評估模型性能
    r2 = r2_score(y_test, y_pred)
    print(f'R-squared in {category}: {r2}')
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error for {category}: {mse}')

R-squared in Music: 0.7630486600375337
Mean Squared Error for Music: 0.018547693423014586
R-squared in Shows: 0.5090030674754216
Mean Squared Error for Shows: 1.4821088596552718
R-squared in News & Politics: 0.5736492392074791
Mean Squared Error for News & Politics: 0.047466267119695656


## Ridge

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
categories = df['category'].unique()
# 初始化一個字典，用於存儲每個類別的模型
models12 = {}
for category in choose:
  category_data = df[df['category'] == category]
  if len(category_data) < 10:
      print(f'Skipping category {category} due to insufficient data.')
      continue
    # 選擇特徵變數和目標變數
  features = category_data[['subscribers', 'video views', 'uploads']]
  target = category_data['mean_monthly_earnings']
  X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
  preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), ['subscribers', 'video views', 'uploads'])
        ])
  # 初始化 Ridge 正規化線性迴歸模型
  ridge_model = Ridge()  # alpha 是正規化項的強度，可以調整

  # 建立管道
  pipeline_ridge = Pipeline([
        ('preprocessor', preprocessor),
        ('model', ridge_model)
    ])

  # 在訓練集上擬合模型
  pipeline_ridge.fit(X_train, y_train)

  # 存儲模型
  models12[category] = pipeline_ridge

  # 預測測試集
  y_pred_ridge = pipeline_ridge.predict(X_test)

  # 評估模型性能
  mse_ridge = mean_squared_error(y_test, y_pred_ridge)
  r2 = r2_score(y_test, y_pred_ridge)
  print(f'R-squared for Ridge in {category}: {r2}')
  print(f'Mean Squared Error for Ridge in {category} : {mse_ridge}')

R-squared for Ridge in Music: 0.7652424216102294
Mean Squared Error for Ridge in Music : 0.01837597370579334
R-squared for Ridge in Shows: 0.5691371988863879
Mean Squared Error for Ridge in Shows : 1.3005897441007035
R-squared for Ridge in News & Politics: 0.5806446469397685
Mean Squared Error for Ridge in News & Politics : 0.046687457926498005


## Lasso回歸

In [ ]:
from sklearn.linear_model import Lasso
categories = df['category'].unique()

# 初始化一個字典，用於存儲每個類別的模型
models13 = {}
for category in choose:
  category_data = df[df['category'] == category]
  if len(category_data) < 10:
      print(f'Skipping category {category} due to insufficient data.')
      continue
    # 選擇特徵變數和目標變數
  features = category_data[['subscribers', 'video views', 'uploads']]
  target = category_data['mean_monthly_earnings']
  X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
  preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), ['subscribers', 'video views', 'uploads'])
        ])
  lasso_model = Lasso()

# 建立管道
  pipeline_lasso = Pipeline([
    ('preprocessor', preprocessor),
    ('model', lasso_model)])

# 在訓練集上擬合模型
  pipeline_lasso.fit(X_train, y_train)
  models13[category] = pipeline_lasso

# 預測測試集
  y_pred_lasso = pipeline_lasso.predict(X_test)

# 評估模型性能
  mse_lasso = mean_squared_error(y_test, y_pred_lasso)
  r2_lasso = r2_score(y_test, y_pred_lasso)
  print(f'R-squared for Lasso in {category}: {r2_lasso}')
  print(f'Mean Squared Error for Lasso in {category}: {mse_lasso}')

R-squared for Lasso in Music: -0.001575836881588577
Mean Squared Error for Lasso in Music: 0.07839973205182804
R-squared for Lasso in Shows: 0.183600413366092
Mean Squared Error for Lasso in Shows: 2.464359714321528
R-squared for Lasso in News & Politics: -0.19043143036252053
Mean Squared Error for Lasso in News & Politics: 0.1325325095145462


## SVM

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import r2_score
categories = df['category'].unique()

# 初始化一個字典，用於存儲每個類別的模型
models_r2_svm = {}

for category in choose:
  category_data = df[df['category'] == category]
  if len(category_data) < 10:
        print(f'Skipping category {category} due to insufficient data.')
        continue

    # 如果該類別樣本數不足，跳過
    # 選擇特徵變數和目標變數
  features = category_data[['subscribers', 'video views', 'uploads']]
  target = category_data['mean_monthly_earnings']

    # 分割數據為訓練集和測試集，並確保保持類別的均衡
  X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # 定義數據預處理的管道
  preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), ['subscribers', 'video views', 'uploads'])
        ])
  svm_model = SVR()

    # 建立管道
  pipeline_svm = Pipeline([
        ('preprocessor', preprocessor),
        ('model', svm_model)
    ])

    # 在訓練集上擬合模型
  pipeline_svm.fit(X_train, y_train)

    # 存儲模型
  models_r2_svm[category] = pipeline_svm

    # 預測測試集
  y_pred_svm = pipeline_svm.predict(X_test)

  # 評估模型性能
  mse_svm = mean_squared_error(y_test, y_pred_svm)
  r2_svm = r2_score(y_test, y_pred_svm)
  print(f'R-squared for svm in {category}: {r2_svm}')
  print(f'Mean Squared Error for SVM in {category}: {mse_svm}')


R-squared for svm in Music: 0.4732592191555819
Mean Squared Error for SVM in Music: 0.041231362177775184
R-squared for svm in Shows: 0.49476110827574105
Mean Squared Error for SVM in Shows: 1.5250992176604896
R-squared for svm in News & Politics: 0.023539502209459884
Mean Squared Error for SVM in News & Politics: 0.10871080594250891


## 共線性問題

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
for category in choose:
  category_data = df[df['category'] == category]
  features = category_data[['subscribers', 'video views', 'uploads']]

# 对特征进行标准化，因为VIF对变量的尺度敏感
  scaler = StandardScaler()
  scaled_features = scaler.fit_transform(features)

# 计算 VIF
  vif_data = pd.DataFrame()
  vif_data["Variable"] = features.columns
  vif_data["VIF"] = [variance_inflation_factor(scaled_features, i) for i in range(scaled_features.shape[1])]

# 打印结果
  print(category)
  print(vif_data)

Music
      Variable       VIF
0  subscribers  5.101255
1  video views  5.253076
2      uploads  1.081033
Shows
      Variable        VIF
0  subscribers  22.787826
1  video views  27.824055
2      uploads   4.826462
News & Politics
      Variable       VIF
0  subscribers  1.490871
1  video views  1.752519
2      uploads  1.232431


可以看到在Shows中有共線性的問題，而在Shows和News & Politics在以上的分數表現都不佳，因此另外用DecisionTreeRegressor做。

## DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# 初始化一個字典，用於存儲每個類別的模型
for category in choose:
  category_data = df[df['category'] == category]
  if len(category_data) < 10:
      print(f'Skipping category {category} due to insufficient data.')
      continue
    # 選擇特徵變數和目標變數
  target = category_data['mean_monthly_earnings']
  features = category_data[['subscribers', 'video views', 'uploads']]
  X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
  preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), ['subscribers', 'video views', 'uploads'])
        ])
  # 初始化決策樹模型
  tree_model = DecisionTreeRegressor()
  pipeline_tree = Pipeline([
    ('preprocessor', preprocessor),
    ('model', tree_model)])
  pipeline_tree.fit(X_train, y_train)
  y_pred_tree = pipeline_tree.predict(X_test)
  r2_tree = r2_score(y_test, y_pred_tree)
  print(f'R-squared for Decision Tree in {category}: {r2_tree}')

  mse_tree = mean_squared_error(y_test, y_pred_tree)
  print(f'MSE for Decision Tree in {category}: {mse_tree}')

R-squared for Decision Tree in Music: -0.40286338176014613
MSE for Decision Tree in Music: 0.1098110689029329
R-squared for Decision Tree in Shows: 0.9699801523768099
MSE for Decision Tree in Shows: 0.0906170266666666
R-squared for Decision Tree in News & Politics: 0.7111614007547299
MSE for Decision Tree in News & Politics: 0.03215683274675001


可以從上面發現 Music 用 Ridge 做分數最高\
Shows 和 News & Politics 用 DecisionTreeRegressor 做最高\
可能原因:Ridge是一種線性模型，而在 Music 這個類別中的變數比較符合線性關係，至於 lasso會有問題可能是因為變數間的關係性較高，而lasso會把係數壓到0導致分數很低。\
而從 vif 中可以發現 Shows 的共線性問題很大，所以用非線性模型做會比較好。\
因為 Shows 和 News & Politics 用 DecisionTreeRegressor 做最高，所以 Shows 和 News & Politics 就可能不是線性關係。\
要注意的是 DecisionTreeRegressor 可能會有過度擬合的問題

## 用交叉驗證檢查過度擬合

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor


for category in choose:
    category_data = df[df['category'] == category]
    if len(category_data) < 10:
        print(f'Skipping category {category} due to insufficient data.')
        continue


    target = category_data['mean_monthly_earnings']
    features = category_data[['subscribers', 'video views', 'uploads']]


    tree_model = DecisionTreeRegressor(random_state=2024)


    pipeline_tree = Pipeline([
        ('preprocessor', preprocessor),
        ('model', tree_model)
    ])

    r2_scores = cross_val_score(pipeline_tree, features, target, cv=5, scoring='r2')
    mse_scores = cross_val_score(pipeline_tree, features, target, cv=5, scoring='neg_mean_squared_error')


    mean_r2 = r2_scores.mean()
    mean_mse = -mse_scores.mean()

    print(f'R-squared for Decision Tree in {category}: {mean_r2}')
    print(f'Mean Squared Error for Decision Tree in {category}: {mean_mse}')

R-squared for Decision Tree in Music: -1.2011594257071274
Mean Squared Error for Decision Tree in Music: 0.2047327792089612
R-squared for Decision Tree in Shows: -56.217032680358514
Mean Squared Error for Decision Tree in Shows: 1.3513682485833327
R-squared for Decision Tree in News & Politics: -0.12156656957719388
Mean Squared Error for Decision Tree in News & Politics: 0.05658769472270062


可以發現使用 DecisionTreeRegressor 會出現過度擬合的問題，所以這個方法在實際上也沒辦法用。

##增加變數

從以上幾個模型可以發現都有一些問題，所以我們這裡考慮增加變數來增進模型，透過 mean_monthly_earnings 和其他變數的 corr 來挑選增加的變數。

In [ ]:
correlation_matrix = df.corr()
correlation_matrix.loc["mean_monthly_earnings"]

<ipython-input-18-040935f9bef5>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = df.corr()


rank                                      -0.248071
subscribers                                0.388601
video views                                0.551494
uploads                                    0.167262
video_views_rank                          -0.208932
country_rank                              -0.148960
channel_type_rank                         -0.187967
video_views_for_the_last_30_days           0.680293
lowest_monthly_earnings                    0.999960
highest_monthly_earnings                   1.000000
lowest_yearly_earnings                     0.999890
highest_yearly_earnings                    0.999958
subscribers_for_last_30_days               0.679681
created_year                               0.073230
created_date                              -0.040402
Gross tertiary education enrollment (%)   -0.061986
Population                                 0.104787
Unemployment rate                         -0.042642
Urban_population                           0.081225
Latitude    

這裡我們增加 video_views_for_the_last_30_days 和 subscribers_for_last_30_days 這兩個變數

In [ ]:
df['video_views_for_the_last_30_days'] = df['video_views_for_the_last_30_days'].fillna(0)
df['subscribers_for_last_30_days'] = df['subscribers_for_last_30_days'].fillna(0)

用 Ridge 做

In [ ]:
categories = df['category'].unique()
# 初始化一個字典，用於存儲每個類別的模型
models12 = {}
for category in choose:
  category_data = df[df['category'] == category]
  if len(category_data) < 10:
      print(f'Skipping category {category} due to insufficient data.')
      continue
    # 選擇特徵變數和目標變數
  features = category_data[['subscribers', 'video views', 'uploads','video_views_for_the_last_30_days','subscribers_for_last_30_days']]
  target = category_data['mean_monthly_earnings']
  X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
  preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), ['subscribers', 'video views', 'uploads','video_views_for_the_last_30_days','subscribers_for_last_30_days'])
        ])
  # 初始化 Ridge 正規化線性迴歸模型
  ridge_model = Ridge()  # alpha 是正規化項的強度，可以調整

  # 建立管道
  pipeline_ridge = Pipeline([
        ('preprocessor', preprocessor),
        ('model', ridge_model)
    ])

  # 在訓練集上擬合模型
  pipeline_ridge.fit(X_train, y_train)

  # 存儲模型
  models12[category] = pipeline_ridge

  # 預測測試集
  y_pred_ridge = pipeline_ridge.predict(X_test)

  # 評估模型性能
  mse_ridge = mean_squared_error(y_test, y_pred_ridge)
  r2 = r2_score(y_test, y_pred_ridge)
  print(f'R-squared for Ridge in {category}: {r2}')
  print(f'Mean Squared Error for Ridge in {category} : {mse_ridge}')

R-squared for Ridge in Music: 0.9063144246849136
Mean Squared Error for Ridge in Music : 0.007333367810362303
R-squared for Ridge in Shows: 0.9234818878078676
Mean Squared Error for Ridge in Shows : 0.23097531673149216
R-squared for Ridge in News & Politics: 0.9977536120287859
Mean Squared Error for Ridge in News & Politics : 0.00025009372868930123


增加變數後確實也增加了R-squared，接下來做一些檢查。

VIF

In [ ]:
for category in choose:
  category_data = df[df['category'] == category]
  features = category_data[['subscribers', 'video views', 'uploads','video_views_for_the_last_30_days','subscribers_for_last_30_days']]

# 对特征进行标准化，因为VIF对变量的尺度敏感
  scaler = StandardScaler()
  scaled_features = scaler.fit_transform(features)

# 计算 VIF
  vif_data = pd.DataFrame()
  vif_data["Variable"] = features.columns
  vif_data["VIF"] = [variance_inflation_factor(scaled_features, i) for i in range(scaled_features.shape[1])]

# 打印结果
  print(category)
  print(vif_data)

Music
                           Variable       VIF
0                       subscribers  5.915155
1                       video views  5.319025
2                           uploads  1.133219
3  video_views_for_the_last_30_days  1.159356
4      subscribers_for_last_30_days  2.453707
Shows
                           Variable        VIF
0                       subscribers  30.794969
1                       video views  87.644201
2                           uploads   5.795107
3  video_views_for_the_last_30_days  57.366437
4      subscribers_for_last_30_days   7.648228
News & Politics
                           Variable       VIF
0                       subscribers  2.178656
1                       video views  2.442024
2                           uploads  1.774553
3  video_views_for_the_last_30_days  4.465913
4      subscribers_for_last_30_days  3.421204


可以看到在Shows中還是有共線性的問題，而其他兩者沒有。

交叉驗證

In [ ]:
from sklearn.model_selection import cross_val_score

choose = ["Music", "Shows", "News & Politics"]
for category in choose:
    # 選擇特定類別的數據
    category_data = df[df['category'] == category]

    # 如果該類別樣本數不足，跳過
    if len(category_data) < 10:
        print(f'Skipping category {category} due to insufficient data.')
        continue

    # 選擇特徵變數和目標變數
    features = category_data[['subscribers', 'video views', 'uploads', 'video_views_for_the_last_30_days', 'subscribers_for_last_30_days']]
    target = category_data['mean_monthly_earnings']

    # 定義數據預處理的管道
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), ['subscribers', 'video views', 'uploads', 'video_views_for_the_last_30_days', 'subscribers_for_last_30_days'])
        ])

    # 初始化線性迴歸模型
    model = Ridge()

    # 建立管道
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)])

    # 使用交叉驗證評估模型性能
    scores = cross_val_score(pipeline, features, target, cv=5, scoring='r2')  # 這裡使用 R-squared 作為評分指標

    # 列印每個折疊的 R-squared 分數
    print(f'R-squared scores for {category}: {scores}')

    # 列印平均 R-squared 分數
    print(f'Mean R-squared for {category}: {scores.mean()}')

R-squared scores for Music: [  0.65194335   0.62822643 -95.20073603   0.52221068   0.62627652]
Mean R-squared for Music: -18.554415811430324
R-squared scores for Shows: [  0.86525449   0.95946996  -0.13708126   0.97390454 -37.30417976]
Mean R-squared for Shows: -6.92852640672874
R-squared scores for News & Politics: [0.97431978 0.96761274 0.98750225 0.98579127 0.96771222]
Mean R-squared for News & Politics: 0.9765876519452311


## 用DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# 初始化一個字典，用於存儲每個類別的模型
for category in choose:
  category_data = df[df['category'] == category]
  if len(category_data) < 10:
      print(f'Skipping category {category} due to insufficient data.')
      continue
    # 選擇特徵變數和目標變數
  target = category_data['mean_monthly_earnings']
  features = category_data[['subscribers', 'video views', 'uploads', 'video_views_for_the_last_30_days', 'subscribers_for_last_30_days']]
  X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
  preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), ['subscribers', 'video views', 'uploads', 'video_views_for_the_last_30_days', 'subscribers_for_last_30_days'])
        ])
  # 初始化決策樹模型
  tree_model = DecisionTreeRegressor()
  pipeline_tree = Pipeline([
    ('preprocessor', preprocessor),
    ('model', tree_model)])
  pipeline_tree.fit(X_train, y_train)
  y_pred_tree = pipeline_tree.predict(X_test)
  r2_tree = r2_score(y_test, y_pred_tree)
  print(f'R-squared for Decision Tree in {category}: {r2_tree}')

  mse_tree = mean_squared_error(y_test, y_pred_tree)
  print(f'MSE for Decision Tree in {category}: {mse_tree}')

R-squared for Decision Tree in Music: 0.9907002661513895
MSE for Decision Tree in Music: 0.0007279495121951223
R-squared for Decision Tree in Shows: 0.983512025295554
MSE for Decision Tree in Shows: 0.04977011416666657
R-squared for Decision Tree in News & Politics: 0.9769566402367057
MSE for Decision Tree in News & Politics: 0.0025654516666666683


交叉檢定

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor


for category in choose:
    category_data = df[df['category'] == category]
    if len(category_data) < 10:
        print(f'Skipping category {category} due to insufficient data.')
        continue


    target = category_data['mean_monthly_earnings']
    features = category_data[['subscribers', 'video views', 'uploads', 'video_views_for_the_last_30_days', 'subscribers_for_last_30_days']]


    tree_model = DecisionTreeRegressor(random_state=2024)


    pipeline_tree = Pipeline([
        ('preprocessor', preprocessor),
        ('model', tree_model)
    ])

    r2_scores = cross_val_score(pipeline_tree, features, target, cv=5, scoring='r2')
    mse_scores = cross_val_score(pipeline_tree, features, target, cv=5, scoring='neg_mean_squared_error')


    mean_r2 = r2_scores.mean()
    mean_mse = -mse_scores.mean()

    print(f'R-squared for Decision Tree in {category}: {mean_r2}')
    print(f'Mean Squared Error for Decision Tree in {category}: {mean_mse}')

R-squared for Decision Tree in Music: -0.7749481835630323
Mean Squared Error for Decision Tree in Music: 0.1773315598788025
R-squared for Decision Tree in Shows: -93.65255065635122
Mean Squared Error for Decision Tree in Shows: 0.8508462426666664
R-squared for Decision Tree in News & Politics: 0.8425085044998578
Mean Squared Error for Decision Tree in News & Politics: 0.013185325045615834


## 小結

從以上的結果發現在News & Politics這個類別中，我們的模型有很高的分數，在Music和Shows中發現有過度擬和的問題，Shows也有共線性的問題。\
代表未來若是要做預測，可能只有在News & Politics這個類別中有比較好的效果。(在變數是這五個的情況下)

# 預測用戶是否會訂閱新的頻道

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# 假設您的目標是預測用戶是否會訂閱新的頻道，這裡創建一個二元的目標變數 'subscribe_future'
df['subscribers_for_last_30_days']=df['subscribers_for_last_30_days'].fillna(0)
df['subscribe_future'] = (df['subscribers_for_last_30_days'] > 20000).astype(int)

# 選擇特徵和目標變數
features = df[['subscribers', 'video views', 'uploads', 'subscribers_for_last_30_days']]
target = df['subscribe_future']

# 對類別特徵進行編碼
label_encoder = LabelEncoder()
df['category_encoded'] = label_encoder.fit_transform(df['category'])

# 加入編碼後的類別特徵
features['category_encoded'] = df['category_encoded']

# 分割數據集
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# 初始化邏輯回歸模型
model = LogisticRegression()

# 訓練模型
model.fit(X_train, y_train)

# 預測
y_pred = model.predict(X_test)

# 評估模型性能
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{report}')

Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        68
           1       1.00      1.00      1.00       131

    accuracy                           1.00       199
   macro avg       1.00      1.00      1.00       199
weighted avg       1.00      1.00      1.00       199



<ipython-input-51-b9edf811e081>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['category_encoded'] = df['category_encoded']


## 查看樣本比例差異

In [ ]:
df['subscribe_future'].value_counts()

1    598
0    397
Name: subscribe_future, dtype: int64

這表示類別 1 的樣本數比類別 0 多。樣本數的不同可能對某些機器學習模型產生影響，特別是對於某些算法來說，較小的類別可能受到較少的關注。

In [ ]:
df['subscribers_for_last_30_days'].value_counts().head()

0.0         337
100000.0    256
200000.0    104
300000.0     59
400000.0     39
Name: subscribers_for_last_30_days, dtype: int64

因為數據的關係，沒有辦法做到1和0的比例接近1:1，所以採用交叉驗證來檢測是否有問題。

## 用交叉驗證

In [ ]:
model1 = LogisticRegression()

# 对整个数据集进行交叉验证，cv参数指定交叉验证的折数
cross_val_scores = cross_val_score(model1, features, target, cv=5, scoring='accuracy')

# 打印每次交叉验证的准确率
for i, score in enumerate(cross_val_scores, 1):
    print(f'Fold {i}: {score}')

# 打印平均准确率
print(f'Mean Accuracy: {cross_val_scores.mean()}')

Fold 1: 0.9396984924623115
Fold 2: 1.0
Fold 3: 1.0
Fold 4: 1.0
Fold 5: 0.9949748743718593
Mean Accuracy: 0.9869346733668343


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


雖然樣本比例有些差異，不過交叉驗證後也沒有太大問題，準確度也很高，可以參考，但樣本比例差異可能是隱憂。

Accuracy (準確度): 準確度是模型正確預測的樣本比例。在這裡，準確度為 99.5%，這表示模型正確預測了 99.5% 的測試樣本。

Classification Report (分類報告):

Precision (精確度):\
對於類別 0（未訂閱新的頻道），精確度為 100%。這表示模型在預測為未訂閱的情況下，有 100% 的機會是正確的。\
對於類別 1（訂閱新的頻道），精確度為 99%。這表示模型在預測為訂閱的情況下，有 99% 的機會是正確的。\
Recall (召回率):\
對於類別 0，召回率為 99%。這表示實際為未訂閱的樣本中，有 99% 被模型正確預測為未訂閱。\
對於類別 1，召回率為 100%。這表示實際為訂閱的樣本中，有 100% 被模型正確預測為訂閱。\
F1-Score (F1 分數):\
F1 分數是精確度和召回率的調和平均值。對於兩個類別，F1 分數都非常高，這是一個良好的指標。\
Support:\
支持數量是每個類別的實際樣本數。\
總的來說，模型在預測用戶是否會訂閱新的頻道方面表現得非常好，具有高精確度和召回率。然而，要注意，這裡的準確度非常高，但在一些情況下，您可能需要更深入地考慮其他指標，特別是當不平衡類別存在時。

## 係數

In [ ]:
coefficients = model.coef_

# 獲取模型的截距
intercept = model.intercept_

# 列印係數和截距
print('Coefficients:', coefficients)
print('Intercept:', intercept)

Coefficients: [[-1.74934978e-06  2.58278738e-10  4.75971748e-05  1.79733383e-03
  -1.89843883e-10]]
Intercept: [3.60807009e-11]


-1.74934978e-06 是 subscribers 特徵的係數。
2.58278738e-10 是 video views 特徵的係數。
4.75971748e-05 是 uploads 特徵的係數。
1.79733383e-03 是 subscribers_for_last_30_days 特徵的係數。
-1.89843883e-10 是 category_encoded 特徵的係數。
3.60807009e-11 是截距。

# 結論

一開始用全部數據去做訂閱，觀看，影片數量來預測收入，但發現結果不好，嘗試分組後再做模型。但分組後發現只用subscribers，video views和uploads這三個變數做模型都不理想，所以加上video_views_for_the_last_30_days 和 subscribers_for_last_30_days 這兩個變數，之後發現只有在News & Politics這個類別中有比較好的效果\
在預測用戶是否會訂閱新的頻道中有很好的效果，但是要注意查看樣本比例差異的問題。